# Redcard Exploratory Data Analysis

This dataset is taken from a fantastic paper that looks to see how analytical choices made by different data science teams on the same dataset in an attempt to answer the same research question affect the final outcome.

[Many analysts, one dataset: Making transparent how variations in analytical choices affect results](https://osf.io/gvm2z/)

The data can be found [here](https://osf.io/47tnc/).



## The Task

Do an Exploratory Data Analysis on the redcard dataset. Keeping in mind the question is the following: **Are soccer referees more likely to give red cards to dark-skin-toned players than light-skin-toned players?**

- Before plotting/joining/doing something, have a question or hypothesis that you want to investigate
- Draw a plot of what you want to see on paper to sketch the idea
- Write it down, then make the plan on how to get there
- How do you know you aren't fooling yourself
- What else can I check if this is actually true?
- What evidence could there be that it's wrong?


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
import mpld3
import numpy as np
import pandas as pd
import os, sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
sns.set_context("poster", font_scale=1.3)

import missingno as msno
import pandas_profiling

import hdbscan
from sklearn.datasets import make_blobs
import time

## About the Data

> The dataset is available as a list with 146,028 dyads of players and referees and includes details from players, details from referees and details regarding the interactions of player-referees. A summary of the variables of interest can be seen below. A detailed description of all variables included can be seen in the README file on the project website. 

> From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2,053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3,147) that these players played under in their professional career (see Figure 1). We created a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.

> -- https://docs.google.com/document/d/1uCF5wmbcL90qvrk_J27fWAvDcDNrO9o_APkicwRkOKc/edit


| Variable Name: | Variable Description: | 
| -- | -- | 
| playerShort | short player ID | 
| player | player name | 
| club | player club | 
| leagueCountry | country of player club (England, Germany, France, and Spain) | 
| height | player height (in cm) | 
| weight | player weight (in kg) | 
| position | player position | 
| games | number of games in the player-referee dyad | 
| goals | number of goals in the player-referee dyad | 
| yellowCards | number of yellow cards player received from the referee | 
| yellowReds | number of yellow-red cards player received from the referee | 
| redCards | number of red cards player received from the referee | 
| photoID | ID of player photo (if available) | 
| rater1 | skin rating of photo by rater 1 | 
| rater2 | skin rating of photo by rater 2 | 
| refNum | unique referee ID number (referee name removed for anonymizing purposes) | 
| refCountry | unique referee country ID number | 
| meanIAT | mean implicit bias score (using the race IAT) for referee country | 
| nIAT | sample size for race IAT in that particular country | 
| seIAT | standard error for mean estimate of race IAT   | 
| meanExp | mean explicit bias score (using a racial thermometer task) for referee country | 
| nExp | sample size for explicit bias in that particular country | 
| seExp |  standard error for mean estimate of explicit bias measure | 



In [ ]:
# Uncomment one of the following lines and run the cell:

# df = pd.read_csv("../data/redcard/redcard.csv.gz", compression='gzip')
# df = pd.read_csv("https://github.com/cmawer/pycon-2017-eda-tutorial/raw/master/data/redcard/redcard.csv.gz", compression='gzip')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
df.dtypes

In [ ]:
all_columns = df.columns.tolist()
all_columns

## What the teams found


### Choices in model features

The following is the covariates chosen for the respective models: 

<img src="figures/covariates.png" width=80%;>


### Choices in modeling

Of the many choices made by the team, here is a small selection of the models used to answer this question:


<img src="figures/models.png" width=80%;>


## Final Results

 - 0 teams: negative effect
 - 9 teams: no significant relationship
 - 20 teams: finding a positive effect

<img src="figures/results.png" width=80%;>

Above image from: http://fivethirtyeight.com/features/science-isnt-broken/#part2


> …selecting randomly from the present teams, there would have been a 69% probability of reporting a positive result and a 31% probability of reporting a null effect. This raises the possibility that many research projects contain hidden uncertainty due to the wide range of analytic choices available to the researchers. -- Silberzahn, R., Uhlmann, E. L., Martin, D. P., Pasquale, Aust, F., Awtrey, E. C., … Nosek, B. A. (2015, August 20). Many analysts, one dataset: Making transparent how variations in analytical choices affect results. Retrieved from osf.io/gvm2z


Images and data from: Silberzahn, R., Uhlmann, E. L., Martin, D. P., Pasquale, Aust, F., Awtrey, E. C., … Nosek, B. A. (2015, August 20). Many analysts, one dataset: Making transparent how variations in analytical choices affect results. Retrieved from osf.io/gvm2z

## Challenge

Before looking below, try to answer some high level questions about the dataset. 


How do we operationalize the question of referees giving more red cards to dark skinned players?
* Counterfactual: if the player were lighter, a ref is more likely to have given a yellow or no card **for the same offense under the same conditions**
* Regression: accounting for confounding, darker players have positive coefficient on regression against proportion red/total card

Potential issues
* How to combine rater1 and rater2? Average them? What if they disagree? Throw it out?
* Is data imbalanced, i.e. red cards are very rare?
* Is data biased, i.e. players have different amounts of play time? Is this a summary of their whole career?
* How do I know I've accounted for all forms of confounding?

**First, is there systematic discrimination across all refs?**

Exploration/hypotheses:
* Distribution of games played
* red cards vs games played
* Reds per game played vs total cards per game played by skin color
* Distribution of # red, # yellow, total cards, and fraction red per game played for all players by avg skin color
* How many refs did players encounter?
* Do some clubs play more aggresively and get carded more? Or are more reserved and get less?
* Does carding vary by leagueCountry?
* Do high scorers get more slack (fewer cards) for the same position?
* Are there some referees that give more red/yellow cards than others?
* how consistent are raters? Check with Cohen's kappa.
* how do red cards vary by position? e.g. defenders get more?
* Do players with more games get more cards, and is there difference across skin color?
* indication of bias depending on refCountry?

## Understand how the data's organized

The dataset is a single csv where it aggregated every interaction between referee and player into a single row. In other words: Referee A refereed Player B in, say, 10 games, and gave 2 redcards during those 10 games. Then there would be a unique row in the dataset that said: 

    Referee A, Player B, 2 redcards, ... 

This has several implications that make this first step to understanding and dealing with this data a bit tricky. First, is that the information about Player B is repeated each time -- meaning if we did a simple average of some metric of we would likely get a misleading result. 

For example, asking "what is the average `weight` of the players?"

In [ ]:
df.height.mean()

In [ ]:
np.mean(df.groupby('playerShort').height.mean())

Doing a simple average over the rows will risk double-counting the same player multiple times, for a skewed average. The simple (incorrect) average is ~76.075 kg, but the average weight of the players is ~75.639 kg. There are multiple ways of doing this, but doing a groupby on player makes it so that so each player gets counted exactly once.

Not a huge difference in this case but already an illustration of some difficulty.

## Tidy Data

Hadley Wickham's concept of a **tidy dataset** summarized as:

>  - Each variable forms a column
>  - Each observation forms a row
>  - Each type of observational unit forms a table

A longer paper describing this can be found in this [pdf](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf).

Having datasets in this form allows for much simpler analyses. So the first step is to try and clean up the dataset into a tidy dataset. 

The first step that I am going to take is to break up the dataset into the different observational units. By that I'm going to have separate tables (or dataframes) for: 

 - players
 - clubs
 - referees
 - countries
 - dyads

## Create Tidy Players Table

In [ ]:
player_index = 'playerShort'
player_cols = [#'player', # drop player name, we have unique identifier
               'birthday',
               'height',
               'weight',
               'position',
               'photoID',
               'rater1',
               'rater2',
              ]

In [ ]:
# Count the unique variables (if we got different weight values, 
# for example, then we should get more than one unique value in this groupby)
all_cols_unique_players = df.groupby('playerShort').agg({col:'nunique' for col in player_cols})

In [ ]:
all_cols_unique_players.head()

In [ ]:
# If all values are the same per player then this should be empty (and it is!)
all_cols_unique_players[all_cols_unique_players > 1].dropna().head()

In [ ]:
# A slightly more elegant way to test the uniqueness
all_cols_unique_players[all_cols_unique_players > 1].dropna().shape[0] == 0

Hooray, our data passed our sanity check. Let's create a function to create a table and run this check for each table that we create.

In [ ]:
def get_subgroup(dataframe, g_index, g_columns):
    """Helper function that creates a sub-table from the columns and runs a quick uniqueness test."""
    g = dataframe.groupby(g_index).agg({col:'nunique' for col in g_columns})
    if g[g > 1].dropna().shape[0] != 0:
        print("Warning: you probably assumed this had all unique values but it doesn't.")
    return dataframe.groupby(g_index).agg({col:'max' for col in g_columns})

In [ ]:
players = get_subgroup(df, player_index, player_cols)
players.head()

In [ ]:
def save_subgroup(dataframe, g_index, subgroup_name, prefix='../data/redcard/raw_'):
    save_subgroup_filename = "".join([prefix, subgroup_name, ".csv.gz"])
    dataframe.to_csv(save_subgroup_filename, compression='gzip')
    test_df = pd.read_csv(save_subgroup_filename, compression='gzip', index_col=g_index)
    # Test that we recover what we send in
    if dataframe.equals(test_df):
        print("Test-passed: we recover the equivalent subgroup dataframe.")
    else:
        print("Warning -- equivalence test!!! Double-check.")

In [ ]:
save_subgroup(players, player_index, "players")

## Create Tidy Clubs Table

Create the clubs table.

In [ ]:
club_index = 'club'
club_cols = ['leagueCountry']
clubs = get_subgroup(df, club_index, club_cols)
clubs.head()

In [ ]:
clubs['leagueCountry'].value_counts()

In [ ]:
save_subgroup(clubs, club_index, "clubs")

## Create Tidy Referees Table

In [ ]:
referee_index = 'refNum'
referee_cols = ['refCountry']
referees = get_subgroup(df, referee_index, referee_cols)
referees.head()

In [ ]:
referees.refCountry.nunique()

In [ ]:
save_subgroup(referees, referee_index, "referees")

## Create Tidy Countries Table

In [ ]:
country_index = 'refCountry'
country_cols = ['Alpha_3', # rename this name of country
                'meanIAT',
                'nIAT',
                'seIAT',
                'meanExp',
                'nExp',
                'seExp',
               ]
countries = get_subgroup(df, country_index, country_cols)
countries.head()

In [ ]:
rename_columns = {'Alpha_3':'countryName'}
countries = countries.rename(columns=rename_columns)
countries.head()

In [ ]:
countries.shape

In [ ]:
save_subgroup(countries, country_index, "countries")

In [ ]:
# Ok testing this out: 
test_df = pd.read_csv("../data/redcard/raw_countries.csv.gz", compression='gzip', index_col=country_index)

In [ ]:
for (_, row1), (_, row2) in zip(test_df.iterrows(), countries.iterrows()):
    if not row1.equals(row2):
        print(row1)
        print()
        print(row2)
        print()
        break

In [ ]:
row1.eq(row2)

In [ ]:
row1.seIAT - row2.seIAT

In [ ]:
countries.dtypes

In [ ]:
test_df.dtypes

In [ ]:
countries.head()

In [ ]:
test_df.head()

Looks like precision error, so I'm not concerned. All other sanity checks pass.

In [ ]:
countries.tail()

In [ ]:
test_df.tail()

## Create separate (not yet Tidy) Dyads Table

This is one of the more complex tables to reason about -- so we'll save it for a bit later. 

In [ ]:
dyad_index = ['refNum', 'playerShort']
dyad_cols = ['games',
             'victories',
             'ties',
             'defeats',
             'goals',
             'yellowCards',
             'yellowReds',
             'redCards',
            ]

In [ ]:
dyads = get_subgroup(df, g_index=dyad_index, g_columns=dyad_cols)

In [ ]:
dyads.head()

In [ ]:
dyads.shape

In [ ]:
dyads[dyads.redCards > 1].head(10)

In [ ]:
save_subgroup(dyads, dyad_index, "dyads")